# Amazon SageMaker, Spotting anomaly through training an ip-insight model

## 1. Introduction

In this sample notebook, you will train, build, and deploy a model using the IP Insights algorithm and Amazon VPC flowlog data. You will query an Athena table and create a dataset for model training. You will perform data transformation on the results from the VPC flowlog data. Train an IP Insights model with this data. Deploy your model to a SageMaker endpoint and ultimately test your model.

## 2. Setup your environment

In [ ]:
# 1. install 
%conda install openjdk -y
%pip install pyspark 
%pip install sagemaker_pyspark
%pip install awswrangler

In [ ]:
# 2. setup, config .. imports

import boto3
import botocore
import os
import sagemaker
import pandas as pd
import awswrangler as wr

from datetime import datetime
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

dt_today = datetime.now()
str_today = dt_today.strftime("%m_%d_%Y_%H_%M_%S")

bucket = sagemaker.Session().default_bucket()
prefix = "sagemaker/ipinsights-vpcflowlogs"
execution_role = sagemaker.get_execution_role()
region = boto3.Session().region_name
seclakeregion = region.replace("-","_")

# check if the bucket exists
try:
    boto3.Session().client("s3").head_bucket(Bucket=bucket)
except botocore.exceptions.ParamValidationError as e:
    print(
        "You either forgot to specify your S3 bucket or you gave your bucket an invalid name!"
    )
except botocore.exceptions.ClientError as e:
    if e.response["Error"]["Code"] == "403":
        print(f"Hey! You don't have permission to access the bucket, {bucket}.")
    elif e.response["Error"]["Code"] == "404":
        print(f"Hey! Your bucket, {bucket}, doesn't exist!")
    else:
        raise
else:
    print(f"Training input/output will be stored in: s3://{bucket}/{prefix}")
print(f"Session timestamp: {str_today}")

## 3. Query and transform VPC Flow Log data

**Prerequisites**

1. Performed steps described in this article: [Analyze VPC Flow Logs with point-and-click Amazon Athena integration](https://aws.amazon.com/blogs/networking-and-content-delivery/analyze-vpc-flow-logs-with-point-and-click-amazon-athena-integration/)
2. Run the paragraph below after replacing your table name in the SQL statement. 

In [ ]:
# 3. query VPC flow logs from VPC flow log athena integration table
#ocsf_df = wr.athena.read_sql_query("SELECT src_endpoint.instance_uid as instance_id, src_endpoint.ip as sourceip FROM amazon_security_lake_table_"+seclakeregion+"_vpc_flow_1_0 WHERE src_endpoint.ip IS NOT NULL AND src_endpoint.instance_uid IS NOT NULL AND src_endpoint.instance_uid != '-' AND src_endpoint.ip != '-'", database="amazon_security_lake_glue_db_us_east_1", ctas_approach=False, unload_approach=True, s3_output=f"s3://{bucket}/unload/parquet/updated/{str_today}")
wr.s3.delete_objects(f"s3://{bucket}/unload/parquet/updated/{str_today}")
ocsf_df = wr.athena.read_sql_query("""
SELECT interface_id, srcaddr FROM [VPC FLOWLOG TABLE NAME] where starts_with(srcaddr, '10.') and starts_with(dstaddr, '10.') ;
""", database="[DATABASE NAME]", ctas_approach=False, unload_approach=False, s3_output=f"s3://{bucket}/unload/parquet/updated/{str_today}")
ocsf_df.head()

## 6. Download image and train IP Insight model

In [ ]:
# 6 setup training data channel and IPInsights algorithm Docker image
training_path = f"s3://{bucket}/{prefix}/training/training_input.csv"

wr.s3.to_csv(ocsf_df, training_path, header=False, index=False)

In [ ]:
from sagemaker.amazon.amazon_estimator import image_uris

image = sagemaker.image_uris.get_training_image_uri(boto3.Session().region_name,"ipinsights")

In [ ]:
# change instance type depending on size of input training
ip_insights = sagemaker.estimator.Estimator(
    image,
    execution_role,
    instance_count=1,
    instance_type="ml.m5.2xlarge",
    output_path=f"s3://{bucket}/{prefix}/output",
    sagemaker_session=sagemaker.Session(),
)

# change hyperparameters depending on size of input training and desired training constraints
ip_insights.set_hyperparameters(
    num_entity_vectors="20000",
    random_negative_sampling_rate="5",
    vector_dim="128",
    mini_batch_size="1000",
    epochs="5",
    learning_rate="0.01",
)

In [ ]:
input_data = {
    "train": sagemaker.session.s3_input(training_path, content_type="text/csv")
}

In [ ]:
# train and fit IPInsights model based on training data

ip_insights.fit(input_data)

## 7. Deploy Sagemaker Endpoint

In [ ]:
# deploy trained IPInsights model to SageMaker endpoint.  Again, change instance_type and autoscaling based on your scenario
predictor = ip_insights.deploy(initial_instance_count=1, instance_type="ml.m5.large")
print(f"Endpoint name: {predictor.endpoint}")

## 8. Submit network data for inference to the endpoint

This portion of code assumes you have test data saved in a local folder or a S3 bucket. 

The test data is simply a CSV file, where the first columns are instance ids and the second columns are IPs. 

It is recommended to test valid and invalid data to see the results of the model

In [ ]:
# read file
# file @ S3 approach
inference_df = wr.s3.read_csv(training_path, header=None).sample(5)

# file @ local approach
anomaly_df = wr.pandas.read_csv('../data/testdata-ipinsights.csv',header=None)
inference_df = pd.concat([inference_df, anomaly_df])

In [ ]:
# prepare bulk request from data frame
import io
from io import StringIO

csv_file = io.StringIO()
inference_csv = inference_df.to_csv(csv_file, sep=",", header=False, index=False)
inference_request_payload = csv_file.getvalue()
print(inference_request_payload)

In [ ]:
# invoke deployed SageMaker model using inference request payload
inference_response = predictor.predict(
    inference_request_payload,
    initial_args={"ContentType":'text/csv'})

# log response
print(inference_response)

## 9.Cleanup

In [ ]:
# delete endpoint if necessary to minimize costs
predictor.delete_endpoint()